<a href="https://colab.research.google.com/github/KamilZPWr/Skyhack-3/blob/main/Patryk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [95]:
from google.colab import drive, files
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install catalyst

In [3]:
from pathlib import Path

import pandas as pd
import numpy as np
import torch
import torch.nn as nn

from PIL import Image
from torch.utils.data import Dataset, DataLoader ,random_split
from torchvision import datasets, models, transforms

from catalyst import dl
from catalyst.dl.callbacks import AccuracyCallback, EarlyStoppingCallback
from catalyst.utils import metrics

In [103]:
ROOT_DIR = '/content/drive/My Drive/Colab Notebooks/'
SUBMISSION_FILE = 'submission.csv'
NUM_CLASSES = 38
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(DEVICE)

cuda


In [70]:
class MultiClassDataset(Dataset):

    def __init__(self , csv_file , img_dir , transform=None):
        self.df = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.transform = transform
    
    def __getitem__(self, idx):
        d = self.df.iloc[idx]
        image = Image.open(f'{self.img_dir}/{d.Name}').convert("RGB")
        label = torch.tensor(d[1:].tolist() , dtype=torch.float32)
    
        if self.transform is not None:
            image = self.transform(image)
        return image, label
  
    def __len__(self):
        return len(self.df)

class TestDataset(Dataset):

    def __init__(self, img_dir, transform):
        self.transform = transform
        self.images = [os.path.join(img_dir, fname) for fname in os.listdir(img_dir)]

    def __getitem__(self, idx):
        image = Image.open(self.images[idx]).convert("RGB")

        if self.transform is not None:
            image = self.transform(image)
        return image
  
    def __len__(self):
        return len(self.images)

In [71]:
batch_size = 16
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
dataset = MultiClassDataset(ROOT_DIR + 'data/training_labels.csv' , ROOT_DIR + 'data/training_images' , transform)
test_set = TestDataset(ROOT_DIR + 'data/live_test_images', transform)

number_of_validation_samples = int(len(dataset)*0.10) 
training_set, validation_set  = random_split(dataset , [len(dataset) - number_of_validation_samples, number_of_validation_samples])

print(f'Number of samples: Train: {len(training_set)}, Validation: {len(validation_set)}')
print(f'Number of samples Test: {len(test_set)}')
loaders = {
    "train":DataLoader(training_set , shuffle=True, batch_size=batch_size),
    "valid": DataLoader(validation_set , shuffle=True, batch_size=batch_size),
    "test": DataLoader(test_set, shuffle=False, batch_size=batch_size)
    }

Number of samples: Train: 3733, Validation: 414
Number of samples Test: 500


In [11]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

def get_model(num_classes, feature_extract=True):
    model = models.alexnet(pretrained = True)
    set_parameter_requires_grad(model, feature_extract)
    num_ftrs = model.classifier[6].in_features
    model.classifier[6] = nn.Linear(num_ftrs, num_classes)
    return model

def skyhacks_f1_score(preds, y):
    return np.mean([metrics.f1_score(preds[:, i], y[:, i])[0].item() for i in range(preds.shape[0])])

In [12]:
model = get_model(NUM_CLASSES)

In [13]:
optimizer = torch.optim.Adam(model.parameters())
criterion = nn.BCEWithLogitsLoss()
callbacks = [
    EarlyStoppingCallback(patience=5)
]
num_epochs = 2

class CustomRunner(dl.Runner):
    
    def predict_batch(self, batch):
        x, y = batch
        return self.model(x.to(self.device))
    
    def _handle_batch(self, batch):
        x, y = batch
        output = self.model(x)
        loss = self.state.criterion(output, y)

        preds = torch.sigmoid(output).data > 0.5
        preds = preds.to(torch.float32)

        f1 = skyhacks_f1_score(preds, y)
        self.batch_metrics = {
            "loss": loss,
            "f1-score": f1
        }
        
        if self.state.is_train_loader:
            loss.backward()
            self.state.optimizer.step()
            self.state.optimizer.zero_grad()           


In [15]:
runner = CustomRunner()

runner.train(
    loaders=loaders,
    model=model, 
    criterion=criterion, 
    optimizer=optimizer,
    callbacks=callbacks,
    num_epochs=num_epochs, 
    logdir="./logs", 
    verbose=True
)

1/2 * Epoch (train): 100% 234/234 [11:28<00:00,  2.94s/it, f1-score=1.000, loss=0.212]
1/2 * Epoch (valid): 100% 26/26 [01:23<00:00,  3.21s/it, f1-score=0.943, loss=0.231]
[2020-11-13 21:53:19,159] 
1/2 * Epoch 1 (train): f1-score=0.9445 | loss=0.1933
1/2 * Epoch 1 (valid): f1-score=0.9341 | loss=0.2086
2/2 * Epoch (train): 100% 234/234 [02:21<00:00,  1.65it/s, f1-score=0.978, loss=0.182]
2/2 * Epoch (valid): 100% 26/26 [00:15<00:00,  1.63it/s, f1-score=0.941, loss=0.243]
[2020-11-13 21:56:03,563] 
2/2 * Epoch 2 (train): f1-score=0.9482 | loss=0.1665
2/2 * Epoch 2 (valid): f1-score=0.9362 | loss=0.2150
Top best models:
logs/checkpoints/train.1.pth	0.2086


In [74]:
results = []

for batch in loaders['test']:
    res = runner.model(batch.to(runner.device))
    res = (torch.sigmoid(res) > 0.5).to(torch.float32).cpu().numpy()
    results.append(res)

In [77]:
df = pd.read_csv(ROOT_DIR + 'data/training_labels.csv')

In [106]:
df_test = pd.DataFrame(np.vstack(results), columns = df.columns[1:], dtype=int)
names = pd.DataFrame({'Name': [Path(el).parts[-1] for el in loaders['test'].dataset.images]})

In [108]:
submission = pd.concat([names, df_test], axis = 1)
submission.to_csv(SUBMISSION_FILE, index = False)
files.download(SUBMISSION_FILE) 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>